In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from skimage import io
import os, sys
import SimpleITK as sitk
from tqdm import tqdm
from os.path import expanduser
HOME = expanduser("~")
%pylab inline

#%matplotlib inline

In [ ]:
DIR = os.path.join(HOME, 'dk39')
ORIENTED = os.path.join(DIR, 'preps', 'oriented')
ALIGNED = os.path.join(DIR, 'preps', 'aligned')
INPUT = ORIENTED

In [ ]:
import cv2
from cv2 import imread, imshow

In [ ]:
def simple_resample(fixed_image, moving_image):
    return  

In [ ]:
# create some empty lists
img_inputs = []
img_outputs = []
file_inputs = []
# the first image is aligned to itself
movingFile = files[0]
file_inputs.append(movingFile)
movingPath = os.path.join(INPUT, movingFile)
fixedPath = movingPath
fixed_image =  sitk.ReadImage(fixedPath, sitk.sitkFloat32)
moving_image = sitk.ReadImage(movingPath, sitk.sitkFloat32) 
fixed_image = sitk.Elastix(fixed_image, moving_image)
img_outputs.append(sitk.GetArrayFromImage(fixed_image))

orient = io.imread(movingPath)
img_inputs.append(orient)

In [ ]:
# loop through the files and create a new image which is then used as the fixed image
for file in tqdm(files[1:4]):
    movingFile = file
    file_inputs.append(movingFile)
    movingPath = os.path.join(INPUT, movingFile)
    moving_image = sitk.ReadImage(movingPath, sitk.sitkFloat32) 
    fixed_image = simple_resample(fixed_image, moving_image)
    img_outputs.append(sitk.GetArrayFromImage(fixed_image))
    orient = io.imread(movingPath)
    img_inputs.append(orient)

In [ ]:
OUTPUT = ALIGNED
dels = os.listdir(OUTPUT)
for d in dels:
    os.unlink(os.path.join(OUTPUT, d))

In [ ]:
for i, img in enumerate(img_outputs):   
    
    filename = '{}_{}'.format(str(i).zfill(4), file_inputs[i])
    outfile = os.path.join(OUTPUT, filename)
    flat = img.flatten()
    fmax = int(flat.max())
    fmin = int(flat.min())
    flat = flat + abs(fmin)
    img = np.reshape(flat, img.shape)    
    img[img <= 0] = 0
    io.imsave(outfile, img.astype('uint16'), check_contrast=False)
    img = None
print('done')

In [ ]:
start, finish = 0,10
for name,file in zip(file_inputs[start:finish], img_outputs[start:finish]):
    plt.style.use('classic')
    plt.figure()
    plt.title('{} {}'.format(name, file.shape))
    plt.imshow(file, cmap='gray')
    plt.show()